In [ ]:
# All of your imports here (you may need to add some)
import numpy
import scipy
import pandas
import matplotlib.pylab as plot
import seaborn

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler, MinMaxScaler

from sklearn import set_config
set_config(transform_output = "pandas")

Frame the Problem and Look at the Big Picture
=====================================
1. **Define the objective in business terms:** Predict if it will be a long winter or a short winter.
2. **How will your solution be used?** 
3. **How should you frame this problem?** <mark>
4. **How should performance be measured? Is the performance measure aligned with the business objective?** 
5. **What would be the minimum performance needed to reach the business objective?** 
6. **What are comparable problems? Can you reuse (personal or readily available) experience or tools?** 
7. **Is human expertise available?**
8. **How would you solve the problem manually?** 
9. **List the assumptions you (or others) have made so far. Verify assumptions if possible.** 